# LangGraph: Control Flow Patterns

## Complete Guide with Executable Examples

### Topics Covered:
1. **Conditional Edges and Routing Logic**
2. **Parallel Execution Strategies**
3. **Sequential Workflows**
4. **Loops and Cycles in Graphs**
5. **Deferred Node Execution** (2025 Feature)

## Setup and Installation

In [3]:
!pip install langgraph langchain-anthropic langchain-core -q

In [4]:
import os
from typing import Annotated, Literal, TypedDict, Optional, Sequence
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.tools import tool
from langchain_anthropic import ChatAnthropic
import operator
from datetime import datetime
import time

os.environ["ANTHROPIC_API_KEY"] = "your-api-key-here"

print("✅ Imports completed successfully!")

✅ Imports completed successfully!


---

# 1. Conditional Edges and Routing Logic

Conditional edges allow dynamic routing based on state.

**Types of Routing:**
- Binary decisions (A or B)
- Multi-way routing (A, B, C, ...)
- Complex state-based routing
- Tool-based routing

## 1.1 Simple Binary Routing

In [7]:
class BinaryState(TypedDict):
    messages: Annotated[list, add_messages]
    needs_approval: bool


def analyze_request(state: BinaryState):
    """Analyze if request needs approval"""
    last_message = state["messages"][-1].content
    
    sensitive_keywords = ["delete", "remove", "drop", "admin"]
    needs_approval = any(keyword in last_message.lower() for keyword in sensitive_keywords)
    
    print(f"🔍 Analyzing: '{last_message[:50]}...'")
    print(f"   Needs approval: {needs_approval}")
    
    return {
        "messages": [AIMessage(content="Request analyzed")],
        "needs_approval": needs_approval
    }


def approve_request(state: BinaryState):
    print("✅ Request approved!")
    return {"messages": [AIMessage(content="Request approved")]}


def execute_directly(state: BinaryState):
    print("⚡ Executing directly")
    return {"messages": [AIMessage(content="Request executed")]}


def route_based_on_approval(state: BinaryState) -> Literal["approve", "execute"]:
    """Router function"""
    if state["needs_approval"]:
        print("🔀 Routing to: approve")
        return "approve"
    else:
        print("🔀 Routing to: execute")
        return "execute"


# Build graph
workflow = StateGraph(BinaryState)
workflow.add_node("analyze", analyze_request)
workflow.add_node("approve", approve_request)
workflow.add_node("execute", execute_directly)

workflow.add_edge(START, "analyze")
workflow.add_conditional_edges(
    "analyze",
    route_based_on_approval,
    {"approve": "approve", "execute": "execute"}
)
workflow.add_edge("approve", END)
workflow.add_edge("execute", END)

binary_app = workflow.compile()

# Test
print("\n🧪 Test 1: Safe request")
result1 = binary_app.invoke({
    "messages": [HumanMessage(content="Get user information")],
    "needs_approval": False
})

print("\n🧪 Test 2: Sensitive request")
result2 = binary_app.invoke({
    "messages": [HumanMessage(content="Delete all records")],
    "needs_approval": False
})


🧪 Test 1: Safe request
🔍 Analyzing: 'Get user information...'
   Needs approval: False
🔀 Routing to: execute
⚡ Executing directly

🧪 Test 2: Sensitive request
🔍 Analyzing: 'Delete all records...'
   Needs approval: True
🔀 Routing to: approve
✅ Request approved!


## 1.2 Multi-Way Routing

In [9]:
class MultiRouteState(TypedDict):
    messages: Annotated[list, add_messages]
    task_type: str
    priority: str


def classify_task(state: MultiRouteState):
    message = state["messages"][-1].content.lower()
    
    if "bug" in message or "error" in message:
        task_type, priority = "bug_fix", "high"
    elif "feature" in message or "add" in message:
        task_type, priority = "feature", "medium"
    elif "question" in message or "help" in message:
        task_type, priority = "support", "low"
    else:
        task_type, priority = "general", "low"
    
    print(f"📋 Task: {task_type} (priority: {priority})")
    return {
        "messages": [AIMessage(content=f"Classified as {task_type}")],
        "task_type": task_type,
        "priority": priority
    }


def handle_bug(state):
    print("🐛 Bug Fix Team")
    return {"messages": [AIMessage(content="Bug assigned")]}

def handle_feature(state):
    print("✨ Product Team")
    return {"messages": [AIMessage(content="Feature added to roadmap")]}

def handle_support(state):
    print("💬 Support Team")
    return {"messages": [AIMessage(content="Support will respond")]}

def handle_general(state):
    print("📝 General Queue")
    return {"messages": [AIMessage(content="Request queued")]}


def route_by_task_type(state: MultiRouteState) -> Literal["bug_fix", "feature", "support", "general"]:
    task_type = state["task_type"]
    print(f"🔀 Routing to: {task_type}")
    return task_type


# Build graph
multi_workflow = StateGraph(MultiRouteState)
multi_workflow.add_node("classify", classify_task)
multi_workflow.add_node("bug_fix", handle_bug)
multi_workflow.add_node("feature", handle_feature)
multi_workflow.add_node("support", handle_support)
multi_workflow.add_node("general", handle_general)

multi_workflow.add_edge(START, "classify")
multi_workflow.add_conditional_edges(
    "classify",
    route_by_task_type,
    {"bug_fix": "bug_fix", "feature": "feature", "support": "support", "general": "general"}
)

for node in ["bug_fix", "feature", "support", "general"]:
    multi_workflow.add_edge(node, END)

multi_app = multi_workflow.compile()

# Test
test_messages = [
    "Critical bug in login",
    "Add export feature",
    "How to reset password?",
    "Just saying hi"
]

for msg in test_messages:
    print(f"\n📨 Input: '{msg}'")
    multi_app.invoke({
        "messages": [HumanMessage(content=msg)],
        "task_type": "",
        "priority": ""
    })


📨 Input: 'Critical bug in login'
📋 Task: bug_fix (priority: high)
🔀 Routing to: bug_fix
🐛 Bug Fix Team

📨 Input: 'Add export feature'
📋 Task: feature (priority: medium)
🔀 Routing to: feature
✨ Product Team

📨 Input: 'How to reset password?'
📋 Task: general (priority: low)
🔀 Routing to: general
📝 General Queue

📨 Input: 'Just saying hi'
📋 Task: general (priority: low)
🔀 Routing to: general
📝 General Queue


---

# 2. Parallel Execution Strategies

Execute multiple nodes simultaneously for:
- Independent data processing
- Multiple API calls
- Parallel validation
- Concurrent computations

## 2.1 Parallel Processing with Merge

In [12]:
class ParallelState(TypedDict):
    input_data: str
    result_a: str
    result_b: str
    result_c: str
    final_result: str


def process_a(state: ParallelState):
    print("⚙️  Process A: Starting...")
    time.sleep(0.5)
    result = f"A processed: {state['input_data']}"
    print(f"   ✅ Process A: Complete")
    return {"result_a": result}


def process_b(state: ParallelState):
    print("⚙️  Process B: Starting...")
    time.sleep(0.5)
    result = f"B processed: {state['input_data']}"
    print(f"   ✅ Process B: Complete")
    return {"result_b": result}


def process_c(state: ParallelState):
    print("⚙️  Process C: Starting...")
    time.sleep(0.5)
    result = f"C processed: {state['input_data']}"
    print(f"   ✅ Process C: Complete")
    return {"result_c": result}


def merge_results(state: ParallelState):
    print("🔄 Merging results...")
    final = f"Combined: {state['result_a']}, {state['result_b']}, {state['result_c']}"
    print(f"   ✅ Merged")
    return {"final_result": final}


# Build parallel graph
parallel_workflow = StateGraph(ParallelState)
parallel_workflow.add_node("process_a", process_a)
parallel_workflow.add_node("process_b", process_b)
parallel_workflow.add_node("process_c", process_c)
parallel_workflow.add_node("merge", merge_results)

# All three processes start from START
parallel_workflow.add_edge(START, "process_a")
parallel_workflow.add_edge(START, "process_b")
parallel_workflow.add_edge(START, "process_c")

# All converge to merge
parallel_workflow.add_edge("process_a", "merge")
parallel_workflow.add_edge("process_b", "merge")
parallel_workflow.add_edge("process_c", "merge")
parallel_workflow.add_edge("merge", END)

parallel_app = parallel_workflow.compile()

# Test
start_time = time.time()
result = parallel_app.invoke({
    "input_data": "test data",
    "result_a": "",
    "result_b": "",
    "result_c": "",
    "final_result": ""
})
end_time = time.time()

print(f"\n⏱️  Execution time: {end_time - start_time:.2f}s")

⚙️  Process A: Starting...
⚙️  Process B: Starting...
⚙️  Process C: Starting...
   ✅ Process A: Complete
   ✅ Process B: Complete
   ✅ Process C: Complete
🔄 Merging results...
   ✅ Merged

⏱️  Execution time: 0.51s


---

# 3. Sequential Workflows

Execute nodes one after another in a pipeline.

## 3.1 Linear Pipeline

In [15]:
class PipelineState(TypedDict):
    raw_data: str
    cleaned_data: str
    processed_data: str
    analyzed_data: str
    final_output: str


def step1_clean(state: PipelineState):
    print("🧹 Step 1: Cleaning...")
    cleaned = state["raw_data"].strip().lower()
    print(f"   '{state['raw_data']}' → '{cleaned}'")
    return {"cleaned_data": cleaned}


def step2_process(state: PipelineState):
    print("⚙️  Step 2: Processing...")
    processed = state["cleaned_data"].replace(" ", "_")
    print(f"   '{state['cleaned_data']}' → '{processed}'")
    return {"processed_data": processed}


def step3_analyze(state: PipelineState):
    print("📊 Step 3: Analyzing...")
    analyzed = f"analyzed({state['processed_data']})"
    print(f"   '{state['processed_data']}' → '{analyzed}'")
    return {"analyzed_data": analyzed}


def step4_finalize(state: PipelineState):
    print("✅ Step 4: Finalizing...")
    final = f"FINAL: {state['analyzed_data']}"
    print(f"   '{state['analyzed_data']}' → '{final}'")
    return {"final_output": final}


# Build pipeline
pipeline_workflow = StateGraph(PipelineState)
pipeline_workflow.add_node("clean", step1_clean)
pipeline_workflow.add_node("process", step2_process)
pipeline_workflow.add_node("analyze", step3_analyze)
pipeline_workflow.add_node("finalize", step4_finalize)

pipeline_workflow.add_edge(START, "clean")
pipeline_workflow.add_edge("clean", "process")
pipeline_workflow.add_edge("process", "analyze")
pipeline_workflow.add_edge("analyze", "finalize")
pipeline_workflow.add_edge("finalize", END)

pipeline_app = pipeline_workflow.compile()

# Test
result = pipeline_app.invoke({
    "raw_data": "  Hello World  ",
    "cleaned_data": "",
    "processed_data": "",
    "analyzed_data": "",
    "final_output": ""
})

print(f"\n📤 Final: {result['final_output']}")

🧹 Step 1: Cleaning...
   '  Hello World  ' → 'hello world'
⚙️  Step 2: Processing...
   'hello world' → 'hello_world'
📊 Step 3: Analyzing...
   'hello_world' → 'analyzed(hello_world)'
✅ Step 4: Finalizing...
   'analyzed(hello_world)' → 'FINAL: analyzed(hello_world)'

📤 Final: FINAL: analyzed(hello_world)


---

# 4. Loops and Cycles in Graphs

Loops allow graphs to repeat nodes until a condition is met.

## 4.1 Simple Loop with Counter

In [18]:
class LoopState(TypedDict):
    count: int
    max_iterations: int
    values: list


def increment_counter(state: LoopState):
    count = state.get("count", 0) + 1
    print(f"🔄 Iteration {count}")
    
    values = state.get("values", [])
    values.append(f"value_{count}")
    
    return {"count": count, "values": values}


def check_continue(state: LoopState) -> Literal["loop", "done"]:
    count = state.get("count", 0)
    max_iter = state.get("max_iterations", 5)
    
    if count < max_iter:
        print(f"🔀 Continue ({count}/{max_iter})")
        return "loop"
    else:
        print(f"🔀 Complete ({count}/{max_iter})")
        return "done"


def finalize_loop(state: LoopState):
    print(f"✅ Loop completed: {len(state['values'])} iterations")
    return state


# Build loop graph
loop_workflow = StateGraph(LoopState)
loop_workflow.add_node("increment", increment_counter)
loop_workflow.add_node("finalize", finalize_loop)

loop_workflow.add_edge(START, "increment")
loop_workflow.add_conditional_edges(
    "increment",
    check_continue,
    {"loop": "increment", "done": "finalize"}
)
loop_workflow.add_edge("finalize", END)

loop_app = loop_workflow.compile()

# Test
result = loop_app.invoke({
    "count": 0,
    "max_iterations": 5,
    "values": []
})

print(f"\n📊 Values: {result['values']}")

🔄 Iteration 1
🔀 Continue (1/5)
🔄 Iteration 2
🔀 Continue (2/5)
🔄 Iteration 3
🔀 Continue (3/5)
🔄 Iteration 4
🔀 Continue (4/5)
🔄 Iteration 5
🔀 Complete (5/5)
✅ Loop completed: 5 iterations

📊 Values: ['value_1', 'value_2', 'value_3', 'value_4', 'value_5']


## 4.2 Iterative Refinement Loop

In [20]:
class RefinementState(TypedDict):
    text: str
    quality_score: float
    iteration: int
    max_iterations: int


def refine_text(state: RefinementState):
    iteration = state.get("iteration", 0) + 1
    text = state["text"]
    
    print(f"✨ Refinement iteration {iteration}")
    
    refined_text = text + " [refined]"
    quality_score = min(0.95, 0.5 + (iteration * 0.15))
    
    print(f"   Quality: {quality_score:.2f}")
    
    return {
        "text": refined_text,
        "quality_score": quality_score,
        "iteration": iteration
    }


def check_quality(state: RefinementState) -> Literal["refine", "accept"]:
    quality = state.get("quality_score", 0)
    iteration = state.get("iteration", 0)
    max_iter = state.get("max_iterations", 5)
    
    if quality >= 0.9:
        print(f"🔀 Quality threshold reached ({quality:.2f})")
        return "accept"
    elif iteration >= max_iter:
        print(f"🔀 Max iterations ({iteration}/{max_iter})")
        return "accept"
    else:
        print(f"🔀 Continue refinement ({quality:.2f})")
        return "refine"


def accept_result(state: RefinementState):
    print(f"✅ Accepted (quality: {state['quality_score']:.2f})")
    return state


# Build refinement loop
refinement_workflow = StateGraph(RefinementState)
refinement_workflow.add_node("refine", refine_text)
refinement_workflow.add_node("accept", accept_result)

refinement_workflow.add_edge(START, "refine")
refinement_workflow.add_conditional_edges(
    "refine",
    check_quality,
    {"refine": "refine", "accept": "accept"}
)
refinement_workflow.add_edge("accept", END)

refinement_app = refinement_workflow.compile()

# Test
result = refinement_app.invoke({
    "text": "Initial draft",
    "quality_score": 0.0,
    "iteration": 0,
    "max_iterations": 5
})

print(f"\n📊 Final: {result['iteration']} iterations, quality={result['quality_score']:.2f}")

✨ Refinement iteration 1
   Quality: 0.65
🔀 Continue refinement (0.65)
✨ Refinement iteration 2
   Quality: 0.80
🔀 Continue refinement (0.80)
✨ Refinement iteration 3
   Quality: 0.95
🔀 Quality threshold reached (0.95)
✅ Accepted (quality: 0.95)

📊 Final: 3 iterations, quality=0.95


---

# 5. Deferred Node Execution (2025 Feature)

Schedule nodes for later execution.

## 5.1 Deferred Processing Pattern

In [23]:
from datetime import datetime, timedelta

class DeferredState(TypedDict):
    task_id: str
    status: str
    deferred_until: Optional[str]
    result: str


def schedule_task(state: DeferredState):
    task_id = state.get("task_id", "task_001")
    deferred_until = (datetime.now() + timedelta(seconds=5)).isoformat()
    
    print(f"📅 Task {task_id} scheduled")
    print(f"   Will execute at: {deferred_until}")
    
    return {"status": "scheduled", "deferred_until": deferred_until}


def check_execution_time(state: DeferredState) -> Literal["defer", "execute"]:
    print("🔍 Checking execution time...")
    is_ready = True  # Simulated for demo
    
    if is_ready:
        print("🔀 Time reached - executing")
        return "execute"
    else:
        print("🔀 Not yet time - deferring")
        return "defer"


def wait_deferred(state: DeferredState):
    print("⏳ Waiting...")
    time.sleep(1)
    return {"status": "waiting"}


def execute_deferred(state: DeferredState):
    print(f"⚡ Executing deferred task")
    result = f"Task {state['task_id']} completed"
    return {"status": "completed", "result": result}


# Build deferred graph
deferred_workflow = StateGraph(DeferredState)
deferred_workflow.add_node("schedule", schedule_task)
deferred_workflow.add_node("wait", wait_deferred)
deferred_workflow.add_node("execute", execute_deferred)

deferred_workflow.add_edge(START, "schedule")
deferred_workflow.add_conditional_edges(
    "schedule",
    check_execution_time,
    {"defer": "wait", "execute": "execute"}
)
deferred_workflow.add_edge("wait", "schedule")
deferred_workflow.add_edge("execute", END)

deferred_app = deferred_workflow.compile()

# Test
result = deferred_app.invoke({
    "task_id": "deferred_001",
    "status": "pending",
    "deferred_until": None,
    "result": ""
})

print(f"\n📊 Status: {result['status']}")
print(f"   Result: {result['result']}")

📅 Task deferred_001 scheduled
   Will execute at: 2026-02-09T00:24:56.170759
🔍 Checking execution time...
🔀 Time reached - executing
⚡ Executing deferred task

📊 Status: completed
   Result: Task deferred_001 completed


---

# Summary

## Key Takeaways:

### 1. Conditional Edges
- Dynamic routing based on state
- Binary, multi-way, complex routing
- Use Literal types for type safety

### 2. Parallel Execution
- Independent operations run simultaneously
- Fan-out/fan-in patterns
- Requires proper state merging

### 3. Sequential Workflows
- Linear pipelines
- Step-by-step processing
- Easy to debug

### 4. Loops and Cycles
- Iterative refinement
- Include exit conditions
- Track iteration counts

### 5. Deferred Execution
- Schedule for later
- Priority-based processing
- Background jobs

## Best Practices:

✅ Always include exit conditions in loops

✅ Use type hints for routing functions

✅ Keep node functions focused

✅ Track state changes for debugging

✅ Handle errors at each node

✅ Use descriptive names

## Anti-Patterns:

❌ Infinite loops without exits

❌ Complex logic in routing

❌ Tight coupling between nodes

❌ Missing error handling